In [37]:
import os 
import glob
from scipy.io import loadmat, savemat
import pickle as pkl

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 

from hampel import hampel
from scipy.signal import medfilt
from scipy.signal import find_peaks

from scipy.signal import resample
from scipy.interpolate import interp1d

from scipy.linalg import svd
from scipy import signal

In [21]:
train_pkls = glob.glob('../../../datasets/Data_27012021_noisy/train/*.pkl')
print ('Train:', len(train_pkls))
val_pkls = glob.glob('../../../datasets/Data_27012021_noisy/val/*.pkl')
print ('Val:', len(val_pkls))
benchmark_pkls = glob.glob('../../../datasets/Data_27012021_noisy/benchmark/*.pkl')
print ('Benchmark:', len(benchmark_pkls))
raw_mats = glob.glob('../../../datasets/Data_27012021_noisy/raw/*.mat')
print ('Raw:', len(raw_mats))

Train: 100
Val: 100
Benchmark: 4
Raw: 10


In [22]:
fin = open(train_pkls[0], 'rb')
train_sample = pkl.load(fin)
fin.close()

fin = open(val_pkls[0], 'rb')
val_sample = pkl.load(fin)
fin.close()

fin = open(benchmark_pkls[0], 'rb')
benchmark_sample = pkl.load(fin)
fin.close()

raw_sample = loadmat(raw_mats[0])

In [23]:
print ('Train Sample Keys')
print (train_sample.keys())
print (train_sample['speed'].min(), train_sample['torque'].min())
print (train_sample['speed'].max(), train_sample['torque'].max())

print ('\n\nVal Sample Keys')
print (val_sample.keys())
print (val_sample['speed'].min(), val_sample['torque'].min())
print (val_sample['speed'].max(), val_sample['torque'].max())

print ('\n\nBenchmark Sample Keys')
print (benchmark_sample.keys())
print (benchmark_sample['speed'].min(), benchmark_sample['torque'].min())
print (benchmark_sample['speed'].max(), benchmark_sample['torque'].max())


print ('\n\nRaw Sample Keys')
print (raw_sample.keys())
print (raw_sample['Speed_Hz'].min(), raw_sample['Torque_PercTnom'].min())
print (raw_sample['Speed_Hz'].max(), raw_sample['Torque_PercTnom'].max())

Train Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'reference_speed', 'torque_time', 'speed_time', 'noisy_current_d', 'noisy_current_q', 'noisy_voltage_d', 'noisy_voltage_q'])
-46.99934659995759 -108.65000728620117
36.30022282079326 223.68110738919583


Val Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'reference_speed', 'torque_time', 'speed_time', 'noisy_current_d', 'noisy_current_q', 'noisy_voltage_d', 'noisy_voltage_q'])
-65.07439460163381 -221.33321791127574
64.59089946035814 81.73566589923782


Benchmark Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'referen

In [24]:
def make_mat(pkl_path):
    fin = open(pkl_path, 'rb')
    data = pkl.load(fin)
    fin.close()
    savemat(pkl_path.replace('pkl', 'mat'), data)
    

for pkl_path in train_pkls + val_pkls + benchmark_pkls:
    make_mat(pkl_path)

In [25]:
def ResampleLinear1D(original, targetLen):
    original = np.array(original, dtype=np.float)
    index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.float)
    index_floor = np.array(index_arr, dtype=np.int) #Round down
    index_ceil = index_floor + 1
    index_rem = index_arr - index_floor #Remain

    val1 = original[index_floor]
    val2 = original[index_ceil % len(original)]
    interp = val1 * (1.0-index_rem) + val2 * index_rem
    assert(len(interp) == targetLen)
    return interp

def smooth(x,window_len,window):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise (ValueError, "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise (ValueError, "Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise (ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


In [43]:
def remove_impulse(sig, thr):
    locs = find_peaks(np.abs(np.max(sig) - sig), threshold=thr)[0]
    if len(locs) == 0:
        locs = find_peaks(sig, threshold=thr)[0]
        
    for loc in locs:
        sig[loc] = (sig[loc-1] + sig[loc+1]) / 2
    return sig

def process_raw_data(mat_path):
    print (mat_path)
    data = loadmat(mat_path)
    
    time = data['Time_VSpdTq'][:, 0]
    speed = data['Speed_Hz'][:, 0]
    torque = data['Torque_PercTnom'][:, 0]
    voltage_d = data['VdRef_V'][: ,0]
    voltage_q = data['VqRef_V'][:, 0]

    current_time = data['Time_I'][:, 0]
    current_d = data['IdMeas_A'][:, 0]
    current_q = data['IqMeas_A'][:, 0]
    
    
    f = interp1d(np.arange(current_time.shape[0]), current_d, 'quadratic')
    
    current_d = ResampleLinear1D(current_d, time.shape[0])
    current_q = ResampleLinear1D(current_q, time.shape[0])
    
    print (time.shape)
    
    idx = hampel(pd.Series(speed), window_size=3, n=3)
    idx += hampel(pd.Series(torque), window_size=3, n=3)
    
    idx += hampel(pd.Series(voltage_d), window_size=3, n=3)
    idx += hampel(pd.Series(voltage_q), window_size=3, n=3)
    
    idx += hampel(pd.Series(current_d), window_size=3, n=3)
    idx += hampel(pd.Series(current_q), window_size=3, n=3)
    
    speed = np.delete(speed, idx)
    torque = np.delete(torque, idx)
    current_d = np.delete(current_d, idx)
    current_q = np.delete(current_q, idx)
    voltage_d = np.delete(voltage_d, idx)
    voltage_q = np.delete(voltage_q, idx)
    time = np.delete(time, idx)
    
#     print (speed.mean(), speed.max())
#     print (torque.mean(), torque.max())
#     print (voltage_d.mean(), voltage_d.max())
#     print (voltage_q.mean(), voltage_q.max())
#     print (current_d.mean(), current_d.max())
#     print (current_q.mean(), current_q.max())
    
    
    
    print ('Time', time.shape, time[1] - time[0])
#     print ('Voltage d', voltage_d.shape, voltage_d.min(), voltage_d.max())
#     print ('Votlage q', voltage_q.shape, voltage_q.min(), voltage_q.max())
#     print ('Speed', speed.shape, speed.min(), speed.max())
#     print ('Torque', torque.shape, torque.min(), torque.max())
#     print ('Current d', current_d.shape, current_d.min(), current_d.max())
#     print ('Current q', current_q.shape, current_q.min(), current_q.max())
    
    w = 51
    w_t = "hamming"
    denoised_speed = smooth(speed, window_len=w, window=w_t)
    denoised_torque = smooth(torque, window_len=w, window=w_t)  
    
    data = {'time': time, 
            'noisy_current_d': current_d, 
            'noisy_current_q': current_q, 
            'noisy_voltage_d': voltage_d, 
            'noisy_voltage_q': voltage_q,
            'noisy_speed': speed,
            'noisy_torque': torque,
            'speed': denoised_speed[w//2:-1*w//2+1],
            'torque': denoised_torque[w//2:-1*w//2+1]
            }
    
    savemat(mat_path.replace('raw', 'real'), data)
    
for mat_path in raw_mats:
    process_raw_data(mat_path)

../../../datasets/Data_27012021_noisy/raw/OFVC4kW_NoLM_SpeedVariations2_20190419_Data.mat
(40928,)
Time (20118,) 0.0039999999999995595
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_NoLM_paliers_0to100Hz_Data.mat
(10224,)
Time (7723,) 0.0039999999999995595
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_LM20Hz_TorqueSteps_20190419_Data.mat
(51175,)
Time (39810,) 0.004
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_LM45Hz_TorqueSteps_20190419_Data.mat
(51034,)
Time (40014,) 0.008
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_LM68Hz_paliers_0to120PercTnom_Data.mat
(10178,)
Time (9006,) 0.004
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_NoLM_SpeedVariations_20190419_Data.mat
(51155,)
Time (43202,) 0.008
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_LM60Hz_TorqueSteps_20190419_Data.mat
(51172,)
Time (44149,) 0.004
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_LM10Hz_paliers_0to120PercTnom_Data.mat
(10231,)
Time (7670,) 0.008
../../../datasets/Data_27012021_noisy/ra

In [33]:
train_real_mats = glob.glob('../../../datasets/Data_27012021_noisy/train_real/*.mat')
print ('Train Real:', len(train_real_mats))

Train Real: 3


In [28]:
for train_real_mat in train_real_mats:
    data = loadmat(train_real_mat)
    
    current_d_clean_statics = []
    for item in data['current_d_clean_statics']:
        current_d_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    current_q_clean_statics = []
    for item in data['current_q_clean_statics']:
        current_q_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    voltage_d_clean_statics = []
    for item in data['voltage_d_clean_statics']:
        voltage_d_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    voltage_q_clean_statics = []
    for item in data['voltage_q_clean_statics']:
        voltage_q_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    speed_clean_statics = []
    for item in data['speed_clean_statics']:
        speed_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    torque_clean_statics = []
    for item in data['torque_clean_statics']:
        torque_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    data['current_d_clean_statics'] = current_d_clean_statics
    data['current_q_clean_statics'] = current_q_clean_statics
    data['voltage_d_clean_statics'] = voltage_d_clean_statics
    data['voltage_q_clean_statics'] = voltage_q_clean_statics
    data['speed_clean_statics'] = speed_clean_statics
    data['torque_clean_statics'] = torque_clean_statics
    
    savemat(train_real_mat, data)

IndexError: invalid index to scalar variable.